# LLM Mastery Align
Code to Align LLM 

In [2]:
# # If requirements.txt is in the same directory as your notebook
# # !pip install -r requirements.txt
# import torch

# # Check if CUDA is available
# print(f"CUDA available: {torch.cuda.is_available()}")

# # Check CUDA version
# if torch.cuda.is_available():
#     print(f"CUDA version: {torch.version.cuda}")
    
#     # Get the current device
#     current_device = torch.cuda.current_device()
    
#     # Get the name of the current device
#     print(f"GPU device: {torch.cuda.get_device_name(current_device)}")
    
#     # Test a small tensor operation on GPU
#     x = torch.tensor([1.0, 2.0, 3.0]).cuda()
#     print(f"Tensor on GPU: {x}")
#     print(f"Device of tensor: {x.device}")
# else:
#     print("No GPU support detected. PyTorch is using CPU only.")

CUDA available: True
CUDA version: 12.1
GPU device: NVIDIA GeForce RTX 3080
Tensor on GPU: tensor([1., 2., 3.], device='cuda:0')
Device of tensor: cuda:0


In [ ]:
# import requests, zipfile, io
# file_url = 'https://ideami.com/llm_align'
# print ('downloading fiels using python')
# response = requests.get(file_url)
# zipfile.ZipFile(io.BytesIO(response.content)).extractall('.')

downloading fiels using python


SSLError: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2580)

In [ ]:
# !pip install flash-attn
# !pip install datasets

## Import libraries

In [8]:
from datetime import datetime
from datasets import load_dataset, load_from_disk
import ipdb
import math
import os, sys
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import transformers
from typing import List, Dict, Union

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32= True

torch.cuda.empty_cache()

torch.set_printoptions(threshold=10000)




## Training parameters

In [52]:
# trainign paramters
batch_size = 1
epochs = 3
lr = 6e-5
lr_warmup_steps = 100
context = 1024
alpha = 0.5 # weigiting for ORPO
prompt_max_size = 512
compile = False
dtype = torch.bfloat16
log_iters = 50

# Hyperparamters
dropout = 0.
grad_clip = 1.0
weight_decay =  0.0

# DEvice
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print ('device: ', device) 


device:  cuda


In [17]:
# logging
project_name = 'aligntest'
wandb_log = True
wandb_project = project_name
wandb_run_name = 'alignment-run'
wandb_run_name = 'alignment-run'
wandb_run_name = 'alignment-run' + datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

if wandb_log:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hassanalam (hassanalam-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [58]:
dataset_path = './data/orpo_dataset9'
dataset_name  = 'mlabonne/orpo-dpo-mix-40k'
tokenizer_path = 'tokenizers/tok16384'
checkpoint_dir = './models/'

# Tokenizer Dataset
tokenizer = transformers.AutoTokenizer.from_pretrained(tokenizer_path)

# Set our interction template
# tokenizer.chat_template = "{% for message in messages%}{% if messge['role]==user'}\n{{'<|user|> + messge['content] + eos_token'}}\{}
tokenizer.chat_template = "{% for message in messages %}{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"

tokenizer.pad_token = tokenizer.eos_token

if os.path.exists(dataset_path):
    dataset = load_from_disk(dataset_path)
else:
    print ('filtering and tokenizing datast')
    dataset = load_dataset(dataset_name, split='all')

    # filter dataset
    # dataset = dataset.filter(lambda r: r["source"] != 'toxic-dpo-v0.2')

    # shorten to     512
    def filter_dataset(examples):
        prompt_length = tokenizer.apply_chat_template(examples['chosen'][:-1], tokenize=True, add_generation_prompt=True, return_tensors='pt').size(-1)
        # prompt_length = tokenizer.apply_chat_template(examples['chosen'][:-1], tokenize=True, add_generation_prompt=True, return_tensors='pt').size(-1)  


        if prompt_length < prompt_max_size:
            return True
        else:
            return False

    # preprocess and tokenize data
    def preprocess_dataset (examples: Union [List, Dict]):
        # select chosen field and releimante last itme
        prompt = [tokenizer.apply_chat_template(item[:-1], tokenize=False, add_generation_prompt=True) for item in examples['chosen']]
        chosen = [tokenizer.apply_chat_template(item, tokenize = False) for item in examples['chosen']]
        rejected = [tokenizer.apply_chat_template(item, tokenize = False) for item in examples['rejected']]

        # tokenizer
        # Hugginf face Dict Format
        # Fielsd: ids, type_id, tokesn, offsets, attention_masks, special_tokens_mask, overflowing
        inputs = tokenizer(prompt, max_length=context, padding='max_length', truncation=True, return_tensors='pt')
        pos_labels = tokenizer(chosen, max_length=context, padding='max_length', truncation=True, return_tensors='pt')
        neg_labels = tokenizer(rejected, max_length=context, padding='max_length', truncation=True, return_tensors='pt')

        # add fields to input
        inputs['positive_input_ids'] = pos_labels['input_ids']
        inputs['postivie_attention_mask'] = pos_labels['attention_mask']

        inputs['negative_input_id'] = neg_labels['input_ids']
        inputs['negative_attenion_mask'] = neg_labels['attention_mask']

        return inputs




    dataset = dataset.filter (filter_dataset)

    # pre process and dataset
    # for multiprocessing : num_processing (32, os.cpu_count())
    dataset = dataset.map(preprocess_dataset, batched = True, num_proc=1, remove_columns=dataset.column_names)

    dataset.save_to_disk(dataset_path)

    

filtering and tokenizing datast


Map:   0%|          | 0/39091 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/39091 [00:00<?, ? examples/s]

AttributeError: 'dict' object has no attribute 'column_names'

./data/orpo_dataset
Loading encoded dataset from disk


['input_ids',
 'token_type_ids',
 'attention_mask',
 'positive_input_ids',
 'positive_attention_mask',
 'negative_input_ids',
 'negative_attention_mask']